# An Analysis of The Global Light Pollution Standards

**Prannaya Gupta (M21404)**

_Done as part of **CS4232**: Data Analytics_

## Table of Contents

## Introduction
A side-effect of technological advancement has been the amount of light pollution in the world today. Ever since Thomas Edison’s revolutionary invention of the light bulb, the world has been thrust into a landscape of light-afflicted skies. According to various studies, around 80% of people live under light pollution-afflicted skies every day, and whilst this may not affect the day-to-day life of an individual, astronomers are very much affected by the sudden illumination in the skies. Even the Singaporean sky is very much damaged by light pollution, with 99.5% of all stars being completely invisible without optical aid<sup>[22]</sup>.

In this project, I aim to analyze the implications of the changes in Light Pollution levels over the past few years, utilising global and local data to find possibly relationships. As shown below, I aim to solve a list of Light Pollution-related questions that utilise alternative data to analyse and interpret patterns, including simple statistical and machine-learning related models.

## Research Questions

### A.	What are locations of minimal light pollution intensity which are optimum for astronomical observation?

While one might think that the best locations are in the middle of wilderness or large water bodies (i.e. the ocean), these locations need to be filtered based on accessibility, especially since locations like the middle of the ocean are not feasible locations for people to assemble to watch and will thus not be a great location for astrophotography.

In order to map this, I can use the absolute night-time light datasets<sup>[4-7]</sup> to analyse locations of minimal light intensity. In order to do the filtering of data, I can use the Base Map<sup>[8]</sup> and Matplotlib’s BaseMap Library<sup>[26]</sup>. This can give us a few optimum points best for this type of analysis.


### B.	How has the light pollution data around the world changed? Which countries are most susceptible to high light pollution in the future? Which countries are lessening in terms of light pollution?

There are many countries that have had rampant increases in Light Pollution over the past few years, while others have made an effort to reduce their already increased Light Pollution. We need to map the data in order to find out which countries are susceptible to the level of problematic light pollution that can be found in cities like Singapore.

Due to the possession of the Time Series Data of night-time light pollution data<sup>[1-3]</sup>, we can easily map locations based on relative changes in absolute light magnitude. I intend to use the geolocation datasets<sup>[9-12]</sup> against the time-series night-time light datasets<sup>[1,17]</sup> to find changes and from there and map it using a possible regression model so as to predict the curve of increase/decrease. This can be used to make predictions and compare to other light pollution levels.


### C. What are the impacts of Light Pollution on the Biodiversity around the area?

High Light Pollution can have a detrimental impact on animals’ survival, hence they are likely to be found in areas with lesser light pollution, which can be adopted as a hypothesis. Another investigation could be what types of animals can be found closer to Light pollution related areas. For example, Moths should ideally be found in high light pollution, but endangered species will likely not be found in similar areas.

Based on the eBird Observation Dataset<sup>[13]</sup>, we can find relative values symbolising the animals’ survival patterns and frequency of observations, thereby relating it to Time Series Night-time Light Data so as to procure information regarding how the birds have moved about in the presence of light pollution. The International Barcode of Life dataset<sup>[14]</sup> is also a logical dataset to not only map frequency patterns but also classify the observations based on existing models and thus finding frequencies of these animals based on Time Series Light Pollution.


### D.	What are the effects and relevance of different types of light facilities on light pollution in the area?

One of the factors claimed culpable for the increased light pollution in Singapore is the extreme number of street lights and traffic lights. Hence, I wish to investigate how the frequency of these lights affect the light pollution around the area. I also want to find possible weights to see how relevant these frequencies are.

With the presence of the Light Facilities Datasets<sup>[15-19]</sup>, we can map this on the Absolute Positional Brightness Dataset<sup>[4-7]</sup> so that we can investigate how it’s affects the actual brightness, whilst we can also investigate how it changes with respect to time by using the Time Series night-light data<sup>[1-3]</sup> and the time based datasets<sup>[15,19]</sup>. This can then be used to compute possible patterns and predictions regarding where the Singapore light pollution may lead up to. To consider Singapore alone, I will be using Geopy<sup>[25]</sup> for the most part and filtering coordinates based on available geographical information.


### E.	What is the relation between the average energy consumption and general demographics in each area (eg Tampines, Jurong etc) and the Light Pollution?

A reason for considering this is the lack of mathematical analysis as to how energy consumption in specific districts affect the light pollution there. General mathematics regarding data in Singapore itself hasn’t been explored, hence using this is a good way of exploring something new. We can model the types of houses, general types of people living there, and the energy consumption based on that and from there, we can test how it affects nearby light pollution.

I intend to use the Economical Datasets that give data regarding the Singapore Energy Consumption statistics<sup>[20]</sup> and the Singapore Resident demographics<sup>[21]</sup>. The usage of the Absolute positional brightness datasets<sup>[4-7]</sup>, filtering Singapore similarly to the previous research question. This can therefore be using in conjunction to find possible patterns.

## Set-Up and Imports

Firstly, we need to perform a simple set-up. This involves the following steps:

1. Import the `autoreload` extension which can allow us to avoid leaving Jupyter at a stagnant file state.
2. Install some libraries, like `np`, `imagecodecs`, `lxml` and `gdal`
     - **Note**: For gdal, which is the Python API Wrapper for the C library GDAL, the installation is a lot more complex than just pip install, hence there is a script. Additional info is provided in the corresponding section
3. Import libraries, which include:
     - Crucial Ones like `random`, `re` and `glob`
     - Mathematica Libraries like `numpy` and `scipy`
     - Data Wrangling Libraries like `pandas` and `lxml`
     - Plotting Libraries like `matplotlib`, `seaborn` and `plotly`
     - Web Scraping Libraries like `bs4` and `requests`
     - Image Processing Libraries like `cv2`, `skimage` and `PIL`
     - Machine Learning Libraries like `sklearn`
     
### Installation Instructions

In this project, I am using the following external libraries:
- `scikit-image`: download using conda if possible.
- `pillow`: predownloaded in conda, install using pip if you are using a default python environment
- `opencv-python`: install using pip in any case
- `lxml`: install using pip if possible
- `geopy`: install using pip or conda
- `geopandas`: install using conda if possible
     - `gdal`: installed with Geopandas.


In [1]:
%load_ext autoreload
%autoreload 2
# to ensure kernel resets when files change around it

In [2]:
import random, time, math, datetime, os, re
from glob import glob
import sys, tarfile, gzip
from pprint import pprint

from IPython.display import clear_output, HTML, Markdown
from IPython import display

import warnings
warnings.filterwarnings("ignore")

import np, numpy
import pandas as pd
from lxml import objectify, etree, html
from io import StringIO, BytesIO

import scipy as sp
from scipy import stats, special

import matplotlib as mpl
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
import seaborn as sns
%matplotlib inline
plt.rcParams["figure.figsize"] = (16.0, 8.0)

from bs4 import BeautifulSoup
import requests, bs4

import cv2

from skimage.color import rgb2gray, gray2rgb
from skimage import filters
from skimage.transform import rescale, resize, downscale_local_mean, swirl
from skimage.filters import threshold_otsu, threshold_local, try_all_threshold
import skimage.io as skio
from PIL import ImageOps
import PIL

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LinearRegression

import folium
from folium import plugins
import plotly.express as px

import geopy, geopandas as gp
from osgeo import gdal

## Definitions
This section pertains to tools created with the express purpose of mathematical and image-based analyses.

Here, you can find the following:
1. Correlation Computational Methods
2. Pandas-based customized data reading functions
3. An Image Class for any Image Processing Concerns
4. A GeoTiff Class to access GeoTiff files and embed it as both a GeoDataFrame and Image

In [3]:
# Functions I painstakingly researched and coded with 2 shots of coffee at 3 am

def gamma(n):
    """
    A Simplified Gamma Function for "powers" of 1/2
    Gamma Function for Integers are given by: Γ(n) = (n-1)!
    Gamma Function for Values ending with 1/2: Γ(n) = 0.5 * 1.5 * ... * (n-1) * √(π)

    Gamma Function is generally given by the following expression
    """
    return np.arange(1 - (n%1), n).prod() * np.pi ** (n%1)

def beta(x, y):
    """
    Actual Beta Function based ont he Simplified Gamma Function above.
    """
    return gamma(x)*gamma(y)/gamma(x+y)

def I(x, a, b):
    if gamma(a) == np.inf or gamma(b) == np.inf or gamma(a+b) == 0: return 0
    if x == 0 or x == 1: return x
    if b == 1: return x ** a
    if a == 1: return 1 - (1-x)**b
    if b > 1: return I(x, a, b-1) + (x**a * (1-x)**(b-1))/((b-1)*beta(a, b-1))
    if a > 1: return I(x, a-1, b) - (x**(a-1) * (1-x)**b)/((a-1)*beta(a-1, b))
    return ((-1)**a) * I(x/(x-1), a, 1-a-b) * beta(a, 1-a-b)


def corr(data):
    norm = data - data.mean(axis=0)
    return norm.prod(axis=1).sum() / np.sqrt((norm**2).sum(axis=0).prod())

def p(r, bound):
    return 2*I((1-abs(r))/2, bound, bound)

def pearson(data):
    r = corr(data)
    return r, p(r, data.shape[0]/2-1)

def pearsonr(x, y):
    x, y = x - x.mean(axis=0), y - y.mean(axis=0)
    r = (x.T @ y).sum(axis=0) / np.sqrt(((x**2).sum(axis=0) * (y**2).sum(axis=0)))
    p_val = 2 * np.vectorize(lambda r: p(r, x.shape[0]/2-1))(r)
    return np.stack((r, p_val), axis=len(r.shape))

In [4]:
def read_csv(loc, *args, **kwargs):
    if re.search(r"[(http(s)?):\/\/(www\.)?a-zA-Z0-9@:%._\+~#=]{2,256}\.[a-z]{2,6}\b([-a-zA-Z0-9@:%_\+.~#?&//=]*)", loc):
        return read_csv(requests.get(loc, allow_redirects=True).content, *args, **kwargs)
    loc = loc.strip("\n")
    if re.search(r"[\n:<>\"/\|?*]", loc):
        return pd.read_csv(StringIO(loc), *args, **kwargs)
    else:
        return pd.read_csv(loc, *args, **kwargs)

In [5]:
class Resize:
    def __init__(self, img):
        self.img = img

    def __call__(self, func):
        return self.img.apply(lambda img: resize(img, func(img.shape)), "Resized Version of "+self.img.name)

    # def local_mean(self, output_shape):
    #     return self.img.apply(lambda img: resize_local_mean(img, output_shape), self.img.name+" resized by local mean "+str(output_shape))


class Plot:
    def __init__(self, img):
        self.img = img

    def __call__(self):
        self.img.show()

    def contour(self, ax=None, title=""):
        self.img.contourplot(ax=ax, title=title)

    def imageReport(self):
        self.img.createImageReport()

class LatLong:
    def __init__(self, img):
        self.h, self.w = img.h, img.w

    def LongfromX(self, x):
        return 360 * x / self.w - 180

    def LatfromY(self, y):
        return - np.arctan(np.sinh((y / self.h) * 2 * np.pi - np.pi)) * 180 / np.pi

    def XfromLong(self, long):
        return (x + 180) * self.w / 360

    def YfromLat(self, lat):
        y = (np.arcsinh(np.tan(lat * np.pi / 180)) + np.pi) * self.h / (2 * np.pi)

class Image:
    def __init__(self, img: np.ndarray, name: str, path: str = ""):
        if name == "":
            name = path.split("/")[-1].split(".")[0]

        self.name = name
        self.title = "Image of " + name
        self.path = path
        self.img = img
        self.isGray = len(self.img.shape) == 2
        self.resize = Resize(self)
        self.plot = Plot(self)
        self.latlong = LatLong(self)

    @classmethod
    def open(cls, path: str, name: str = ""):
        return cls(skio.imread(path), name=name, path=path)

    @classmethod
    def read(cls, path:str, name:str = ""):
        if re.search(r"[(http(s)?):\/\/(www\.)?a-zA-Z0-9@:%._\+~#=]{2,256}\.[a-z]{2,6}\b([-a-zA-Z0-9@:%_\+.~#?&//=]*)", path) and re.search("((http|https)://)(www.)?[a-zA-Z0-9@:%._\\+~#?&//=]{2,256}\\.[a-z]{2,6}\\b([-a-zA-Z0-9@:%._\\+~#?&//=]*)", path):
            return cls(np.asarray(PIL.Image.read(requests.get(url, stream=True).raw)), name=name, path=path)
        else: return cls(skio.imread(path), name=name, path=path)

    def show(self, title="", ax=None):
        if ax is None:
            plt.axis("off")
            plt.imshow(self.img, cmap="gray" if self.isGray else None)
            plt.title(title if title else self.title)
            plt.show()
        else:
            plt.axis("off")
            ax.imshow(self.img, cmap="gray" if self.isGray else None)
            ax.set_title(title if title else self.title)

        return "%s - Retrieved from %s with Shape %s" % (self.title, self.path, self.shape)

    def __repr__(self):
        return self.show()

    def __str__(self):
        return self.name

    def to_numpy(self):
        return self.img

    @property
    def shape(self):
        return self.img.shape

    @property
    def h(self):
        return self.img.shape[0]

    @property
    def w(self):
        return self.img.shape[1]

    def gray(self):
        return self if self.isGray else Image(rgb2gray(self.img), name="Grayscale of "+self.name, path=self.path)

    def red(self):
        img = self.img.copy()
        img[:, :, 1:] = 0
        return Image(img, name="Red Part of "+self.name, path=self.path)

    def green(self):
        img = self.img.copy()
        img[:, :, ::2] = 0
        return Image(img, name="Green Part of "+self.name, path=self.path)

    def blue(self):
        img = self.img.copy()
        img[:, :, :2] = 0
        return Image(img, name="Blue Part of "+self.name, path=self.path)

    def flipud(self):
        return Image(np.flipud(self.img), name=self.name+" Flipped Vertically", path=self.path)

    def fliplr(self):
        return Image(np.fliplr(self.img), name=self.name+" Flipped Horizontally", path=self.path)

    def invert(self):
        return Image((1 if np.all(self.img < 1) else 63 if np.all(self.img < 64) else 255) - self.img, name=self.name+" Inverted", path=self.path)

    def transpose(self):
        return Image(np.transpose(self.img, [1, 0, 2]), name=self.name+" Transposed", path=self.path)

    def spotlight(self):
        img = np.copy(self.img)
        h, w = img.shape
        img[np.sqrt(np.arange(-(w-1)/2, (w+1)/2)**2 + np.arange(-(h-1) /
                    2, (h+1)/2)[:, np.newaxis] ** 2) > min(h, w)/2] = 0
        return Image(img, name="Spotlight of "+self.name, path=self.path)

    def convolve(self, kernel):
        image = self.img
        return Image((np.transpose(np.transpose(image[np.arange(kernel.shape[1]) + np.arange(image.shape[0] + 1 - kernel.shape[0])[:, np.newaxis]], (0, 2, 1))[:, np.arange(kernel.shape[0]) + np.arange(image.shape[1] + 1 - kernel.shape[1])[:, np.newaxis]], (0, 1, 3, 2)) * kernel).sum((2, 3)), name=self.name+" Convolved", path=self.path)

    def blur(self, patchSize):
        return Image(self.convolve(kernel=np.full((patchSize, patchSize), 1/patchSize**2)).to_numpy(),
                     name="{}% Blur of {}".format(patchSize, self.name), path=self.path)

    def colorHist(self, title="", ax=None):
        if ax is None:
            plt.hist(self.img.ravel(), bins=256)
            plt.title(title if title else "Color Histogram of %s" % self.name)
            plt.show()
        else:
            ax.hist(self.img.ravel(), bins=256)
            ax.set_title(
                title if title else "Color Histogram of %s" % self.name)

    def threshColorHist(self, thresh, title="", ax=None):
        if ax is None:
            fig, ax = plt.subplots(1, 1)

        self.colorHist(
            ax=ax, title=self.title if self.title else "Thresholded Color Histogram of %s" % self.name)
        try:
            thresh = float(thresh)
            ax.axvline(x=thresh, color="r")
        except:
            pass

    def laplacian(self):
        return Image(cv2.Laplacian(self.img, cv2.CV_64F), "Laplacian Filter on "+self.name, path=self.path)

    def sobelX(self):
        return Image(cv2.Sobel(self.img, cv2.CV_64F, 1, 0, ksize=5), "Sobel X Filter on "+self.name, path=self.path)

    def sobelY(self):
        return Image(cv2.Sobel(self.img, cv2.CV_64F, 0, 1, ksize=5), "Sobel Y Filter on "+self.name, path=self.path)

    def sobel(self):
        return Image(filters.sobel(self.gray().img), "Sobel Filter on "+self.name, path=self.path)

    def toColor(self):
        return Image(cv2.cvtColor(self.img, cv2.COLOR_GRAY2RGB), "Colorized "+self.name, path=self.path) if self.isGray else self

    def rescale(self, scale):
        return self.apply(lambda img: rescale(img, scale), self.name + " Rescaled to {.2f}%".format(scale*100))

    def swirl(self, rotation=0, strength=10, radius=120, *args, **kwargs):
        return self.apply(lambda img: swirl(img, rotation=rotation, strength=strength, radius=radius, *args, **kwargs), self.name+" Swirled")

    def contourplot(self, ax=None, title=""):
        if ax is None:
            plt.contour(self.img, [50, 200])
            plt.title(title if title else "Contourplot of %s" % self.title)
        else:
            ax.contour(self.img, [50, 200])
            ax.set_title(title if title else "Contourplot of %s" % self.title)

    def apply(self, func, name=""):
        return Image(func(self.img), name=name if len(name) else "Function on "+self.name, path=self.path)

    def threshold(self, func, name=""):
        return Image(self.img > func(self.img), name=name if len(name) else "Thresholding Function on "+self.name, path=self.path)

    def scattered(self):
        edges_img = self.sobel().img
        x, y = np.argwhere(edges_img > 0.07).T
        return y, -x

    def scatter(self, ax=None, s=0.1, title="", figsize=(10, 13)):
        x, y = img.scattered()
        if ax is None:
            plt.figure(figsize=figsize)
            plt.scatter(x, y, s=s)
            plt.title(title if title else "Scatterplot of %s" % self.title)
        else:
            ax.scatter(x, y, s=s)
            ax.set_title(title if title else "Scatterplot of %s" % self.title)

        return stats.pearsonr(x, y)

    def createImageReport(self):
        img = self.toColor() if self.isGray else self
        gray = self if self.isGray else self.gray()

        fig, axes = plt.subplots(4, 4, figsize=(30, 30))

        img.show(title="Original "+img.title, ax=axes[0, 0])
        img.flipud().show(ax=axes[0, 1])
        img.fliplr().show(ax=axes[0, 2])
        img.transpose().show(ax=axes[0, 3])

        img.red().show(ax=axes[1, 0])
        img.blue().show(ax=axes[1, 1])
        img.green().show(ax=axes[1, 2])
        img.invert().show(ax=axes[1, 3])

        img.laplacian().show(ax=axes[2, 0])
        img.sobelX().show(ax=axes[2, 1])
        img.sobelY().show(ax=axes[2, 2])
        img.sobel().show(ax=axes[2, 3])

        gray.threshold(
            threshold_otsu, name="Global Thresholding of "+img.name).show(ax=axes[3, 0])
        gray.spotlight().show(ax=axes[3, 1])
        gray.blur(15).show(ax=axes[3, 2])
        gray.colorHist(ax=axes[3, 3])


class GeoTiff(gp.GeoDataFrame):
    def __init__(self, ds, name):
        self.ds = ds
        self.img = ds.GetRasterBand(1).ReadAsArray().astype(np.uint8)
        self.cols = ds.RasterXSize
        self.rows = ds.RasterYSize
        self.shape = np[self.rows, self.cols]
        self.geotransform = np.array(ds.GetGeoTransform())
        self.xmin = geotransform[0]
        self.xmax = np[1, cols] @ geotransform[:2]
        self.xbounds = np[self.xmin, self.xmax]
        self.ymin = np[1, rows] @ geotransform[3:6:2]
        self.ymax = geotransform[3]
        self.ybounds = np[self.ymin, self.ymax]
        self.centerx = self.xbounds.mean()
        self.centery = self.ybounds.mean()
        self.center = np[self.centerx, self.centery]

        path = ds.getDescription()

        self.df = pd.DataFrame(self.img, index=np.linspace(self.ymin, self.ymax, self.rows), columns=np.linspace(self.xmin, self.xmax, self.cols)).stack().replace(0, np.nan).dropna().reset_index().rename(columns={"level_1":"Longitude", "level_0":"Latitude", 0: "Intensity"})

        gp.GeoDataFrame.__init__(self.df, geometry=gp.points_from_xy(self.df.Longitude, self.df.Latitude))
        #Image.__init__(self, img=img, name=name, path=path)

    @classmethod
    def open(cls, path: str, name: str = ""):
        return cls(gdal.Open(path, gdal.GA_ReadOnly), name=name)

    @classmethod
    def read(cls, path: str, name: str = ""):
        return cls(gdal.Open(path, gdal.GA_ReadOnly), name=name)

    def folium(self):
        m = folium.Map(location=self.center, zoom_start=1, tiles='Stamen Terrain')
        incidents = folium.map.FeatureGroup()
        pd.DataFrame.apply(self, lambda row: incidents.add_child(folium.CircleMarker([row.Latitude, row.Longitude], radius=5, color='yellow', fill=True, fill_color='blue', fill_opacity=0.6)) and folium.Marker([row.Latitude, row.Longitude], popup=f"<span style=\"color:#0000FF\">{row.Intensity}</span>").add_to(m), axis = 1)
        m.add_child(incidents)
        return m

## Data Acquisition
I will be acquiring the following datasets in this project:
1. [A harmonized global nighttime light dataset 1992–2018](https://www.nature.com/articles/s41597-020-0510-y)
2. [Globe At Night - Sky Brightness Monitoring Network (GaN-MN)](https://www.globeatnight.org/maps.php)
3.

### Acquring Harmonized Global Nighttime Light (1992 - 2018) Dataset
This dataset is specifically the largest, containing close to 20 billion data points (specifically 20322960028), hence this data has to be acquired in chunks. All the data can be downloaded from a zip file as shown below, and I suggest running the following script on Git Bash, WSL or the Terminal. In case these are not at one's disposal, here are the instructions:
1. Download the zip file from https://figshare.com/ndownloader/articles/9828827/versions/2
2. Make a directory relative to this notebook called `data/nightLight`
3. Unzip all the contents in the zip file into the `nightLight` directory as created in step 2.
4. Remove the zip file, so as to conserve disk space.

Installation can take place as follows:
```sh
mkdir -p data/nightLight
curl https://figshare.com/ndownloader/articles/9828827/versions/2 > nightLight.zip
unzip nightLight.zip -d data/nightLight/
rm nightLight.zip
```

In [7]:
ds = gdal.Open('data/nightLight/Harmonized_DN_NTL_1992_calDMSP.tif', gdal.GA_ReadOnly)
img = ds.GetRasterBand(1).ReadAsArray().astype(np.uint8)
shape = np[ds.RasterYSize, ds.RasterXSize]
print("Shape:", tuple(shape))
geotransform = np.array(ds.GetGeoTransform())
xbounds = np[geotransform[0], np[1, shape[-1]] @ geotransform[:2]]
print("X Bounds:", tuple(xbounds))
ybounds = np[np[1, shape[0]] @ geotransform[3:6:2], geotransform[3]]
print("Y Bounds:", tuple(ybounds))
center = np[xbounds.mean(), ybounds.mean()]
print("Center:", tuple(center))
index = np.linspace(*ybounds, shape[0], dtype=np.uint8)
columns = np.linspace(*xbounds, shape[1], dtype=np.uint8)
df = pd.DataFrame(img, index=index, columns=columns, dtype=np.uint8)
df = df.stack().reset_index()
df = df.rename(columns={"level_1":"Longitude", "level_0":"Latitude", 0: "Intensity"})
gdf = gp.GeoDataFrame(df, geometry=gp.points_from_xy(df.Longitude, df.Latitude))
gdf

Shape: (16801, 43201)
X Bounds: (-180.00416666665, 180.00416522665)
Y Bounds: (-65.00416610665, 75.00416666665)
Center: (-7.200000027296483e-07, 5.000000279999995)


MemoryError: Unable to allocate 1.35 GiB for an array with shape (725820001,) and data type int16

In [ ]:
#img = GeoTiff.read('data/nightLight/Harmonized_DN_NTL_1992_calDMSP.tif', "1992 Light Pollution")
img

In [ ]:
data1992 = img.to_numpy()
latlong = pd.DataFrame(data1992).rename(index=img.latlong.LatfromY, columns=img.latlong.LongfromX).stack()
#.reset_index().rename(columns={"level_1":"Longitude", "level_0":"Latitude", 0: "Pollution"})
latlong = latlong[latlong > 0].reset_index().rename(columns={"level_1":"Longitude", "level_0":"Latitude", 0: "Pollution"})
latlong #.describe()

In [ ]:
latlong.describe()


In [ ]:
ax = plt.subplot(projection='3d')
X, Y = np.meshgrid(latlong.columns, latlong.index)
ax.plot_surface(X, Y, latlong, rstride=1, cstride=1,
                cmap='viridis', edgecolor='none')

In [ ]:
sns.scatterplot(x="Longitude", y="Latitude", hue="Pollution", data=latlong)


### Acquiring the Globe At Night Dataset
Globe At Night collects data based on specific locations, and in this case, contains a column named `LimitingMag` which can be related to Light Pollution standards in the region. The following commands showcase a way to download the dataset programmatically, while also removing unnecessary datasets.

In [ ]:
gan_url = "https://www.globeatnight.org/"
files = [gan_url + i["href"] for i in BeautifulSoup(requests.get(gan_url+"maps.php").content, "lxml").findAll(href=re.compile("\.csv$"))]
!mkdir -p data/gan
gan = []
for file in files:
    print(file)
    filename = "data/gan/"+file.split("/")[-1]
    file = BytesIO(requests.get(file, allow_redirects=True).content)
    data = pd.read_csv(file, error_bad_lines=False)[["Latitude", "Longitude", "LocalDate", "LocalTime", "UTDate", "UTTime", "LimitingMag", "Country"]]
    data = data[data.LimitingMag > 0]
    data.LocalTime = pd.to_datetime(data.apply(lambda row: row["LocalDate"] + " " + row["LocalTime"], axis=1), format='%Y-%m-%d %H:%M')
    data.UTTime = pd.to_datetime(data.apply(lambda row: row["UTDate"] + " " + row["UTTime"], axis=1), format='%Y-%m-%d %H:%M')
    data = data[["Latitude", "Longitude", "LocalTime", "UTTime", "LimitingMag", "Country"]]
    data.to_csv(filename)
    gan.append(data)

gan[0]

In [ ]:
gan[0].Country.unique() #.groupby("UTTime").LimitingMag.mean().plot()

### Acquiring the iBOL Dataset
iBOL is also a pretty important dataset which collects animal images and maps them to locations, hence showing possible locations of certain animals to be found. This also requires downloading the 

In [ ]:
!curl https://hosted-datasets.gbif.org/ibol/ibol.zip > ibol.zip
!mkdir -p data/ibol/
!unzip ibol.zip -d data/ibol/
!rm data/ibol.zip

xml_data = html.parse('data/ibol/meta.xml').getroot().getchildren()[0].getchildren()[0].getchildren()
xml_data = xml_data[:1] + xml_data[3:]
ibol_meta = {}
for el in xml_data:
    el = html.document_fromstring(etree.tostring(el))
    ibol_meta[el.xpath("//files")[0].getchildren()[0].text] = [i.get("term").split("/")[-1] for i in el.xpath("//field")]

pprint(ibol_meta)

In [ ]:
!cat ibol/occurrences.txt | head -n 10
ibol_occ = pd.read_csv("data/ibol/occurrences.txt", delimiter="\t", names=ibol_meta["occurrences.txt"])
ibol_occ

In [ ]:
!cat ibol/media.txt | head -n 10
ibol_media = pd.read_csv("data/ibol/media.txt", delimiter="\t", names=ibol_meta["media.txt"])
ibol_media

In [ ]:
!cat ibol/sequence.txt | head -n 10
ibol_seq = pd.read_csv("data/ibol/sequence.txt", delimiter="\t", names=ibol_meta["sequence.txt"])
ibol_seq

In [ ]:
dmsp_ols_url = "https://ngdc.noaa.gov"
dmsp_ols = requests.get(dmsp_ols_url+"/eog/dmsp/downloadV4composites.html").content
files = [dmsp_ols_url + i["href"].replace("\n", "") for i in BeautifulSoup(dmsp_ols, "lxml").find_all("table", class_="list")[0].find_all("a")]
!rm -r dmsp
!mkdir dmsp_tar dmsp_tif dmsp
for file in files:
  print(file)
  filename = "dmsp_tar/"+file.split("/")[-1]
  r = requests.get(file, allow_redirects=True)
  with open(filename, 'wb') as fileobj: fileobj.write(r.content)

!for f in dmsp_tar/*.tar; do tar xf "$f" -C dmsp_tif/; done
!rm dmsp_tar/*.tar
!gunzip -r dmsp_tif
!rm dmsp/*.gz
output.clear()

GBIF.org (11 September 2021) GBIF Occurrence Download  https://doi.org/10.15468/dl.vkf37t

### Acquiring Lat-Long Dataset For Countries Worldwide
This dataset is pretty explanatory, and quite possibly the easiest to acquire. I have simply read Google Developers' `countries.csv` public dataset.

In [ ]:
countries_latlong = pd.read_html(requests.get("https://developers.google.com/public-data/docs/canonical/countries_csv").content)[0].set_index("name")
countries_latlong

### Acquiring NASA's EaN Blue Marble 2016 Dataset
Link: [https://visibleearth.nasa.gov/images/144898/earth-at-night-black-marble-2016-color-maps](https://visibleearth.nasa.gov/images/144898/earth-at-night-black-marble-2016-color-maps)

Satellite images of Earth at night—often referred to as “night lights”—have been a curiosity for the public and a tool of fundamental research for at least 25 years. They have provided a broad, beautiful picture, showing how humans have shaped the planet and lit up the darkness. Produced every decade or so, such maps have spawned hundreds of pop-culture uses and dozens of economic, social science, and environmental research projects.

These images show Earth’s night lights as observed in 2016. The data were reprocessed with new compositing techniques that select the best cloud-free nights in each month over each land mass.

The images are available as JPEG and GeoTIFF, in three different resolutions: 0.1 degrees (`3600x1800`), 3km (`13500x6750`), and 500m (`86400x43200`). The 500m global map is divided into tiles (21600x21600) according to a gridding scheme.

In [ ]:
# Black Marble
blackMarble2016 = list(np.vectorize("https://eoimages.gsfc.nasa.gov/images/imagerecords/144000/144898/BlackMarble_2016_{}_geo.tif".format)(np.array(["A", "B", "C", "D"], dtype=object)[:, np.newaxis] + np.array(["1", "2"], dtype=object)).flatten())
blackMarble2016

### Acquiring Global Radiance Calibrated Nighttime Lights Dataset
The Global Radiance Calibrated Nighttime Lights, hereby referred to as the RadCal dataset, contains multiple GeoTiff files saved as `.tgz`, hence the task is to first unzip this tar file and then open the GeoTiff Images. The code below contains the exact method to install this.

In [ ]:
urls = [a["href"] for a in BeautifulSoup(requests.get("https://ngdc.noaa.gov/eog/dmsp/download_radcal.html").content).find_all("table")[1].find_all("a", href=True) if a["href"].startswith("https")][::2]
pprint(urls)
!mkdir -p data/radcal
radcal = []
for url in urls:
    print(url)
    fn = "data/radcal/"+url.split("/")[-1]
    with open(fn, "wb") as file: file.write(requests.get(url).content)
    tar = tarfile.open(fn, "r")
    for file in tar:
        tar.extract(file, ".".join(fn.split(".")[:-1])+"/")
!ls data/radcal

### Acquiring Geonames Datasets
Geonames is used to get simple country demographics. While we are only really considering Singapore on a microscopic level, this is used especially to segment the dataset.

In [ ]:
geocountries = pd.read_html(requests.get("https://www.geonames.org/countries/").content)[1]
geocountries.Continent.replace(np.nan, "NA", inplace=True)
geocountries

In [ ]:
geostats = pd.read_html(requests.get("https://www.geonames.org/statistics/").content)[1].loc[:250]
geostats